In [1]:
import pandas as pd
import numpy as np
import cobra
import re
import os

from troppo.omics.readers.generic import TabularReader
from troppo.methods_wrappers import ModelBasedWrapper, ReconstructionWrapper
from troppo.omics.integration import ContinuousScoreIntegrationStrategy
from troppo.methods.reconstruction.imat import IMAT, IMATProperties

<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/home/biodata/miniconda3/envs/cplex_aman_new/lib/python3.10/site-packages/cobamp/wrappers/external_wrappers.py:9: UserWarning: 
The wrappers.external_wrappers module will be deprecated in a future release in favour of the wrappers module. 
    Available ModelObjectReader classes can still be loaded using cobamp.wrappers.<class>. An appropriate model 
    reader can also be created using the get_model_reader function on cobamp.wrappers
  warnings.warn(


In [2]:
from cobamp.core.linear_systems import get_default_solver

print("COBAMP default solver:", get_default_solver())

COBAMP default solver: CPLEX


In [3]:
# parsing rule 
patt = re.compile('__COBAMPGPRDOT__[0-9]{1}') # e.g __COBAMPGPRDOT__2
replace_alt_transcripts = lambda x: patt.sub('', x) #  empty string (prune)

In [4]:
# load model and expression data
model = cobra.io.read_sbml_model('/home/biodata/aman/Human-GEM/model/Human-GEM.xml')
expression_data = pd.read_csv('/home/biodata/aman/data_processed/expression_data_hippocampus-development-human-brain-10XV2_annotated_gencode.csv', index_col=0)

In [5]:
expression_data

,expression
gencode_id,
ENSG00000223972,0.000000
ENSG00000227232,0.372200
ENSG00000278267,0.014134
ENSG00000243485,0.047114
ENSG00000284332,0.000000
...,...
ENSG00000198727,1548.072900
ENSG00000210195,2.289735
ENSG00000210196,0.829204


In [6]:
print(expression_data.head())

expression_data_transposed = expression_data.T

print("\nTransposed shape:", expression_data_transposed.shape)
print(expression_data_transposed.head())

omics_container = TabularReader(path_or_df=expression_data_transposed, 
                                nomenclature='gene',
                                omics_type='transcriptomics').to_containers()

single_sample = omics_container[0]

                 expression
gencode_id                 
ENSG00000223972    0.000000
ENSG00000227232    0.372200
ENSG00000278267    0.014134
ENSG00000243485    0.047114
ENSG00000284332    0.000000

Transposed shape: (1, 35159)
gencode_id  ENSG00000223972  ENSG00000227232  ENSG00000278267  \
expression              0.0           0.3722         0.014134   

gencode_id  ENSG00000243485  ENSG00000284332  ENSG00000237613  \
expression         0.047114              0.0              0.0   

gencode_id  ENSG00000240361  ENSG00000186092  ENSG00000233750  \
expression              0.0              0.0         0.315663   

gencode_id  ENSG00000222623  ...  ENSG00000210184  ENSG00000210191  \
expression              0.0  ...              0.0         0.014134   

gencode_id  ENSG00000198786  ENSG00000198695  ENSG00000210194  \
expression         304.5536        20.494543         0.164899   

gencode_id  ENSG00000198727  ENSG00000210195  ENSG00000210196  \
expression        1548.0729         2.289735

In [7]:
model_wrapper = ReconstructionWrapper(model=model, ttg_ratio=9999,
                                      gpr_gene_parse_function=replace_alt_transcripts)

data_map = single_sample.get_integrated_data_map(model_reader=model_wrapper.model_reader,
                                                 and_func=min, or_func=sum)

/home/biodata/miniconda3/envs/cplex_aman_new/lib/python3.10/site-packages/cobamp/gpr/core.py:41: UserWarning: Could not normalize this rule: _ENSG00000147576 or _ENSG00000172955 or _ENSG00000180011 or _ENSG00000187758 or _ENSG00000196344 or _ENSG00000196616 or _ENSG00000197894 or _ENSG00000198099 or _ENSG00000248144
  warnings.warn('Could not normalize this rule: ' + rule)
/home/biodata/miniconda3/envs/cplex_aman_new/lib/python3.10/site-packages/cobamp/gpr/core.py:41: UserWarning: Could not normalize this rule: _ENSG00000117448
  warnings.warn('Could not normalize this rule: ' + rule)
/home/biodata/miniconda3/envs/cplex_aman_new/lib/python3.10/site-packages/cobamp/gpr/core.py:41: UserWarning: Could not normalize this rule: _ENSG00000131069
  warnings.warn('Could not normalize this rule: ' + rule)
/home/biodata/miniconda3/envs/cplex_aman_new/lib/python3.10/site-packages/cobamp/gpr/core.py:41: UserWarning: Could not normalize this rule: _ENSG00000111058 or _ENSG00000154930
  warnings.war

In [8]:
def score_apply(reaction_map_scores):
    return {k:0  if v is None else v for k, v in reaction_map_scores.items()}

continuous_integration = ContinuousScoreIntegrationStrategy(score_apply=score_apply)
scores = continuous_integration.integrate(data_map=data_map)

In [9]:
scores

{'MAR03905': 104.84724735899998,
 'MAR03907': 109.34191,
 'MAR04097': 6.2048993,
 'MAR04099': 17.5169444,
 'MAR04108': 6.2048993,
 'MAR04133': 6.2048993,
 'MAR04281': 553.81909,
 'MAR04388': 554.247826408,
 'MAR04283': 1.6725430700000001,
 'MAR08357': 240.054694,
 'MAR04379': 141.030721,
 'MAR04301': 141.030721,
 'MAR04355': 63.815762502,
 'MAR04358': 203.93718833399998,
 'MAR04360': 0,
 'MAR04363': 385.1795896,
 'MAR04365': 219.739182167,
 'MAR04368': 171.40033,
 'MAR04370': 73.285656,
 'MAR04371': 219.739182167,
 'MAR04372': 73.285656,
 'MAR04373': 1301.125370836,
 'MAR04375': 310.075342502,
 'MAR04377': 0.659594413,
 'MAR04381': 43.65102,
 'MAR04391': 297.51477,
 'MAR04394': 59.019568924999994,
 'MAR04396': 31.382564000000002,
 'MAR04521': 33.342499556,
 'MAR06412': 16.805525,
 'MAR07745': 6.671327,
 'MAR07747': 33.00799,
 'MAR08360': 56.494263,
 'MAR08652': 33.342499556,
 'MAR08757': 0.89987534,
 'MAR03989': 25.545149950000003,
 'MAR04122': 33.48384,
 'MAR04837': 0.8951638860000001

In [10]:
reaction_ids = model_wrapper.model_reader.r_ids
exp_vector = np.array([scores[rid] for rid in reaction_ids])
exp_vector

array([104.84724736, 109.34191   ,   6.2048993 , ...,   4.6689863 ,
         4.6689863 ,   4.6689863 ])

In [11]:
os.environ["COBAMP_SOLVER"] = "CPLEX"

In [12]:
import optlang
from optlang.cplex_interface import Model as CPLEXModel

optlang.config = {'solver': 'cplex'}


In [13]:
import os
os.environ["OPTLANG_DEFAULT_SOLVER"] = "CPLEX"

In [14]:
print("Available solvers:", optlang.available_solvers)

Available solvers: {'GUROBI': False, 'GLPK': True, 'MOSEK': False, 'CPLEX': True, 'COINOR_CBC': False, 'SCIPY': True, 'OSQP': False, 'HIGHS': False}


In [15]:
# Create the properties for the IMAT algorithm.
properties = IMATProperties(exp_vector=exp_vector, exp_thresholds=(25,75))

# Run the iMAT algorithm.
imat = IMAT(S=model_wrapper.S, lb=model_wrapper.lb, ub=model_wrapper.ub, properties=properties)

model_imat = imat.run()

Solution was not optimal


In [ ]:
properties

exp_vector = [104.84724736 109.34191      6.2048993  ...   4.6689863    4.6689863
   4.6689863 ]
exp_thresholds = (25, 75)
tolerance = 1e-08
epsilon = 1

In [ ]:
pd.DataFrame(model_imat)

,0
0,0
1,1
2,5
3,6
4,9
...,...
9732,12964
9733,12967
9734,12968
9735,12969


In [ ]:
# variable will contain the indices of the reactions that should be kept in the final model. 

In [ ]:
print(type(model_imat))

<class 'numpy.ndarray'>


In [ ]:
pd.DataFrame(model_imat).to_csv('troppo_output_imat.tsv', sep='\t')

---

In [ ]:
# model.reactions
# len(model.reactions)

In [ ]:
selected_reactions = [model.reactions[i] for i in model_imat.flatten().tolist()]
selected_reactions

[<Reaction MAR03905 at 0x7509955abe20>,
 <Reaction MAR03907 at 0x7509955a8070>,
 <Reaction MAR04133 at 0x75099546d600>,
 <Reaction MAR04281 at 0x75099546c370>,
 <Reaction MAR08357 at 0x75099546eaa0>,
 <Reaction MAR04379 at 0x75099546f160>,
 <Reaction MAR04301 at 0x75099546f7c0>,
 <Reaction MAR04355 at 0x75099546fa90>,
 <Reaction MAR04358 at 0x75099546fd00>,
 <Reaction MAR04360 at 0x750995490310>,
 <Reaction MAR04363 at 0x7509955abfd0>,
 <Reaction MAR04365 at 0x750995490790>,
 <Reaction MAR04368 at 0x750995491450>,
 <Reaction MAR04371 at 0x750995491b40>,
 <Reaction MAR04373 at 0x750995492800>,
 <Reaction MAR04375 at 0x750995492500>,
 <Reaction MAR04381 at 0x750995493580>,
 <Reaction MAR04391 at 0x7509954933d0>,
 <Reaction MAR07745 at 0x7509954b8d90>,
 <Reaction MAR08360 at 0x7509954b8e50>,
 <Reaction MAR08652 at 0x7509954b95a0>,
 <Reaction MAR03989 at 0x7509954ba1a0>,
 <Reaction MAR05395 at 0x7509954ba290>,
 <Reaction MAR05396 at 0x7509954bb3a0>,
 <Reaction MAR05397 at 0x7509954bbfd0>,


---

In [ ]:
ctx_model = model.copy()

selected_ids = [r.id for r in selected_reactions]
to_remove = [r for r in ctx_model.reactions if r.id not in selected_ids]

ctx_model.remove_reactions(to_remove, remove_orphans=True)

ctx_model

Name,HumanGEM
Memory address,7509154a0760
Number of metabolites,7792
Number of reactions,9737
Number of genes,2174
Number of groups,148
Objective expression,0
Compartments,"Cytosol, Extracellular, Endoplasmic reticulum, Mitochondria, Peroxisome, Golgi apparatus, Lysosome, Nucleus, Inner mitochondria"


In [ ]:
len(model.reactions), len(ctx_model.reactions)

(12971, 9737)

In [ ]:
# export
cobra.io.write_sbml_model(ctx_model, "tinit_context_specific_model.xml")